In [ ]:
from transformers import ElectraModel, ElectraTokenizerFast
from sklearn.metrics.pairwise import cosine_similarity
from model import modelWithPooling
import pandas as pd
import numpy as np
import torch


model = ElectraModel.from_pretrained("../model/disc_book_final")
tokenizer = ElectraTokenizerFast.from_pretrained("monologg/koelectra-base-v3-discriminator")

model_with_pooling = modelWithPooling(model, tokenizer).to('mps')


In [ ]:
data =  pd.read_csv('../data/bookList/pre_book_total_128.csv',index_col=0)['sen'].values

In [ ]:
from datasets import Dataset
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
dataset = Dataset.from_pandas(pd.DataFrame(data, columns=['sen']))

def tokenizing_function(batch) :
    lst = []
    for i in batch :
        lst.append(i['sen'])

    return tokenizer(lst,truncation=True,padding=True,return_tensors='pt')
    

dataloader = DataLoader(dataset,32,collate_fn=tokenizing_function)

model_with_pooling.eval()

lst = []
for num, item in enumerate(tqdm(dataloader)) :
    item.to('mps')
    x = model_with_pooling(**item)['sentence_embedding'].detach()
    lst.append(x)
    if num % 1000 == 0 and num != 0 :    
        torch.save(torch.cat(lst),f'save/embedding_{num}.pt')
        lst.clear()